In [1]:
from src.classifiers.tuning.hptuning import Tuner
from src.signals.ppg import PPGExperiment
from src.signals.eda import EDAExperiment
from src.classifiers.tuning.hptuning import best_trial_hyperparameter
from src.experiments.experiment import ExperimentType, Experiment
from src.data.utils import wipe_results
from src.signals.eeg import EEGSubject
from src.signals.ppg import PPGSubject
import os
import neurokit2 as nk
from src.classifiers.fcn import Fcn
from src.classifiers.cnn import Cnn
from src.classifiers.lstm import Lstm
from src.classifiers.resnet import ResNet
import pandas as pd
import numpy as np

path = os.getcwd()
survey_path = "{0}/survey_gamification".format(os.getcwd())

2024-09-19 20:08:36.659257: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 20:08:36.659634: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 20:08:36.661744: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-19 20:08:36.667900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 20:08:36.677823: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
wipe_results()

In [ ]:
from src.data_prepare import prepare_data
prepare_data(survey_path)

In [2]:
eeg = EEGSubject(path=os.path.join(path, 'survey_gamification'), subject_id="11")

TypeError: EEGSubject.__init__() got an unexpected keyword argument 'subject_id'

In [2]:
e_fcn = EDAExperiment(classifier="fcn", type=ExperimentType.END_TO_END, path=path)
e_cnn = EDAExperiment(classifier="cnn", type=ExperimentType.END_TO_END, path=path)
e_lstm = EDAExperiment(classifier="lstm", type=ExperimentType.END_TO_END, path=path)

In [2]:
from src.experiments.analysis import Analysis

In [3]:
oh_boy = Experiment(signal="eeg", classifier="fcn", type=ExperimentType.END_TO_END, path=path, device="muse", subject=EEGSubject)

In [4]:
doomer = Tuner(oh_boy)
doomer.tune(max_evals=1)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

2024-09-19 20:08:52,235 [INFO] Running objective for cnn at 0 iteration
2024-09-19 20:08:52,236 [INFO] Experiment for eeg signal, classifier: cnn, fold: 4
job exception: Kernel shape must have the same length as input, but received kernel of shape (7, 1, 12) and input of shape (None, 20, 7680, 1).



  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]


ValueError: Kernel shape must have the same length as input, but received kernel of shape (7, 1, 12) and input of shape (None, 20, 7680, 1).

In [3]:
a = Analysis(type=ExperimentType.END_TO_END, path=path, device="samsung", signal="ppg")

In [ ]:
a.head(3)

In [ ]:
a.compare()

In [4]:

empatica = Analysis(type=ExperimentType.END_TO_END, path=path, device="empatica", signal="ppg")

In [ ]:
empatica.head(3)

In [ ]:
empatica.compare()

In [20]:
aha = a.list_metrics()["fcn"]["df"].head(1)["subject"].values[0]

In [ ]:
aha

In [ ]:
a.head(3, skip=["resnet"])

In [9]:
tuner = Tuner(e_fcn)

In [ ]:
tuner.best_hyperparameters().dict()

In [11]:
aha = {}
aha["aha"] =  42

In [82]:
files = {"fcn": {}, "cnn": {}, "lstm": {}, "resnet":{}}
for file in os.listdir(exp.test_metrics_path):
  if not os.path.isdir(os.path.join(exp.test_metrics_path, file)):
    if file.startswith("fcn"):
      files["fcn"][file[3:5]] = os.path.join(exp.test_metrics_path, file)
    if file.startswith("cnn"):
      files['cnn'][file[3:5]] = os.path.join(exp.test_metrics_path, file)
    if file.startswith("lstm"):
      files['lstm'][file[4:6]] = os.path.join(exp.test_metrics_path, file)
    if file.startswith("resnet"):
      files['lstm'][file[6:8]] = os.path.join(exp.test_metrics_path, file)

In [84]:
df = pd.DataFrame()
for classifier in files:
  df_classifier = pd.DataFrame()
  for subject in files[classifier]:
    metrics_path = files[classifier][subject]
    df_sub = pd.read_csv(metrics_path)
    df_classifier = pd.concat([df_classifier,df_sub.assign(subject=subject, classifier=classifier)])
  files[classifier]["df"] = df_classifier
    


In [ ]:
for c in files:
  if isinstance(files[c]["df"], pd.DataFrame) and 'f1' in files[c]["df"]:
    print(files[c]["df"].sort_values(by="f1", ascending=False).head(3))

In [ ]:
files["fcn"]["df"].sort_values(by="f1", ascending=False).head(3)

In [ ]:
df.groupby("classifier").filter(lambda x: True)

In [ ]:
df.loc("classifier", "f1")

In [71]:
grouped = df.groupby(["classifier"])["f1"].nlargest(3).to_frame().reset_index().drop("level_1", axis=1)

In [ ]:
grouped

In [ ]:
df.filter(lambda s: s).apply(lambda s: s["f1"].nlargest(3), 1)

In [2]:
classifiers = ["resneta"]

In [ ]:
for c in classifiers:
  print(c)
  # exp = PPGExperiment(classifier=c, type=ExperimentType.END_TO_END, path=path, device="samsung")
  # tuner = Tuner(exp)
  # exp.losocv_run(tuner)

  # exp = PPGExperiment(classifier=c, type=ExperimentType.END_TO_END, path=path, device="empatica")
  # tuner = Tuner(exp)
  # exp.losocv_run(tuner)

  exp = EDAExperiment(classifier=c, type=ExperimentType.END_TO_END, path=path)
  tuner = Tuner(exp)
  exp.losocv_run(tuner)

In [ ]:
for c in classifiers:
  print(c)
  exp = PPGExperiment(classifier=c, type=ExperimentType.END_TO_END, path=path, device="samsung")
  tuner = Tuner(exp)
  exp.test_best_models(tuner)

  # exp = PPGExperiment(classifier=c, type=ExperimentType.END_TO_END, path=path, device="empatica")
  # tuner = Tuner(exp)
  # exp.test_best_models(tuner)

  # exp = EDAExperiment(classifier=c, type=ExperimentType.END_TO_END, path=path)
  # tuner = Tuner(exp)
  # exp.test_best_models(tuner)


In [2]:
eda = EDAExperiment(classifier="fcn", type=ExperimentType.END_TO_END, path=path)
eda_tuner = Tuner(eda)

In [ ]:
metrics = eda.test_best_models(eda_tuner)

In [ ]:
metrics = exp.test_best_models(tuner)

In [ ]:
for s in metrics:
  print(s, metrics[s])
  print('')